In [29]:
from google.colab import files
fread = lambda path: open(path,'r').read()

In [ ]:
#@title Day 1: Secret Entrance
import math

inputs = fread('input.txt').splitlines()

def rotate(pos, d, rotation, part1, part2):
  flip = lambda p : [p%100,(100-p)%100][d=='R']
  pos = flip(pos) + rotation
  return (flip(pos), part1 + (flip(pos) == 0), part2 + math.floor(pos / 100))

acc = (50, 0, 0)
[acc := rotate(acc[0], i[0],int(i[1:]), acc[1], acc[2]) for i in inputs]

print(acc[1])
print(acc[2])

997
5978


In [ ]:
#@title Day 2: Gift Shop
import math, re

inputs = fread('input 2.txt')
inputs = [list(map(int,s.split('-'))) for s in inputs.split(',')]

def bads(lower, upper, pattern):
  some = [i for i in range(lower, upper+1) if re.search(pattern, str(i))]
  return sum([s for s in some if s >= lower and s <= upper])

print(sum([bads(*i,r'^([0-9]+)\1$') for i in inputs]))
print(sum([bads(*i,r'^([0-9]+)\1+$') for i in inputs]))

5398419778
15704845910


In [ ]:
#@title Day 3: Lobby
from itertools import accumulate
import math

inputs = fread('input 3.txt')
inputs = [list(map(int,s)) for s in inputs.splitlines()]

def joltage(line, d):
  def best_idx(prev,j):
    offset = [0,prev+1][prev > -1]
    values = line[offset:len(line)-(d-1-j)]
    return max(range(len(values)), key=values.__getitem__)+offset

  idxs = list(accumulate(range(-1,d), best_idx))[1:]
  return sum([int(math.pow(10,d-1-j)) * line[idxs[j]] for j in range(d)])

print(sum(map(lambda l: joltage(l, 2), inputs)))
print(sum(map(lambda l: joltage(l, 12), inputs)))

17207
170997883706617


In [ ]:
#@title Day 4: Printing Department

inputs = """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@."""
inputs = fread('input 4.txt')
inputs = inputs.splitlines()

max_x = len(inputs[0])
max_y = len(inputs)
is_tp = lambda a,x,y: 0 <= x < max_x and 0 <= y < max_y and a[y][x] == '@'

def prune_tp(prev):
  next = []
  count = 0

  for y in range(len(prev)):
    next.append([])

    for x in range(len(prev[0])):
      if not is_tp(prev,x,y):
        next[y].append(prev[y][x])
        continue

      ct = sum([is_tp(prev,i,j) for i in range(x-1,x+2) for j in range(y-1,y+2) if x != i or y != j])

      next[y].append('x' if ct < 4 else '@')
      count += ct < 4

  return (count, next)

def prune_all_tp():
  cont = True
  curr = inputs
  total = 0
  while cont:
    (removed, curr) = prune_tp(curr)
    total += removed
    cont = removed != 0
  return (total, curr)

print(prune_tp(inputs)[0])
print(prune_all_tp()[0])




1564
9401


In [ ]:
#@title Day 5-1: Cafeteria
inputs = [s.splitlines() for s in fread('input 5.txt').split('\n\n')];
ranges = [list(map(int,s.split('-'))) for s in inputs[0]]
ids = list(map(int,inputs[1]))
count = sum([any(low <= id <= high for (low,high) in ranges) for id in ids])
print(count)

640


In [ ]:
#@title Day 5-2: Cafeteria
inputs = fread('input 5.txt')

inputs = [s.splitlines() for s in inputs.split('\n\n')]
ranges = [list(map(int,s.split('-'))) for s in inputs[0]]
ranges = sorted(ranges, key=lambda r: r[0])

def insert(sorted_list, new_element, i, len):
  if(i >= len):
    return sorted_list + [new_element];
  if(sorted_list[i][0] > new_element[1]):
    return sorted_list[:i] + [new_element] + sorted_list[i:]
  if(sorted_list[i][1] < new_element[0]):
    return insert(sorted_list, new_element, i+1, len)
  return sorted_list[:i] + [[
      min(sorted_list[i][0],new_element[0]),
      max(sorted_list[i][1],new_element[1])
      ]] + sorted_list[i+1:]

merged_ranges = []
for i in range(len(ranges)):
  merged_ranges = insert(merged_ranges, ranges[i], 0, len(merged_ranges))

count = sum([high+1-low for (low,high) in merged_ranges])
print(count)

365804144481581


In [ ]:
#@title Day 6: Trash Compactor
from functools import reduce
from operator import add,mul

inputs = fread('input 6.txt')
inputs = inputs.splitlines()

def total(ops, ns): return sum([reduce(o, ns[i]) for i,o in enumerate(ops)])

# Part 1
strs = [s.split() for s in inputs]
operations = [[mul,add][c=='+'] for c in strs[-1]]
nums = [[int(strs[y][x]) for y in range(len(strs)-1)] for x in range(len(strs[0]))]
print(total(operations,nums))

# Part 2
operations = operations[::-1]
strs = [''.join([inputs[y][x] for y in range(len(inputs)-1)]).strip() for x in range(len(inputs[0])-1,-1,-1)]
strs = '\n'.join(strs).split('\n\n')
nums = [list(map(int, s.splitlines())) for s in strs]
print(total(operations, nums))

3785892992137
7669802156452


In [ ]:
#@title Day 7: Laboratories

inputs = fread("input 7.txt")
inputs = inputs.splitlines()

def split(ct, f, s):
  d = {i: 0 for i in range(-1,len(s)+1)}
  for i,c in enumerate(s):
    for j in ([i],[i-1,i+1])[c=='^']: d[j] += f[i]
  return (ct + sum([c=='^' and f[i]>0 for i,c in enumerate(s)]),
   [d[i] for i in range(len(f))])

state = (0, [int(c == 'S') for c in inputs[0]])
[state := split(*state, inputs[i]) for i in range(1,len(inputs))]

print(state[0])
print(sum(state[1]))

1543
3223365367809


In [35]:
#@title Day 8:
import numpy as np

inputs = """162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689"""

def dist1(a,b):
  return np.linalg.norm(a-b)

def dist2(a,b):
  return np.sqrt(np.sum((a-b)**2))

def dist3(a,b):
  return sum([(a[i]-b[i])**2 for i in range(3)])**0.5

def dist(a,b):
  print("dist1",dist1(a,b))
  print("dist2",dist2(a,b))
  print("dist3",dist3(a,b))
  return dist3(a,b)

jns = [np.array(list(map(int, j.split(',')))) for j in inputs.splitlines()]
dists = {(i,j): dist(jn,jns[j]) for i,jn in enumerate(jns) for j in range(i+1,len(jns))}
ordered_dists = sorted(dists, key=lambda kvp: dists[kvp])

# print(dist(jns[0],jns[19]))
# print(dist(jns[5],jns[19]))

dist1 787.814064357828
dist2 787.814064357828
dist3 787.814064357828
dist1 908.7843528582565
dist2 908.7843528582565
dist3 908.7843528582565
dist1 561.7187908553532
dist2 561.7187908553532
dist3 561.7187908553532
dist1 723.7879523727927
dist2 723.7879523727927
dist3 723.7879523727927
dist1 736.432617419951
dist2 736.432617419951
dist3 736.432617419951
dist1 1047.4349621814235
dist2 1047.4349621814235
dist3 1047.4349621814235
dist1 321.560258738545
dist2 321.560258738545
dist3 321.560258738545
dist1 693.2055972076394
dist2 693.2055972076394
dist3 693.2055972076394
dist1 391.46519640959144
dist2 391.46519640959144
dist3 391.46519640959144
dist1 693.0959529531247
dist2 693.0959529531247
dist3 693.0959529531247
dist1 1044.5022738127477
dist2 1044.5022738127477
dist3 1044.5022738127477
dist1 709.0486584149215
dist2 709.0486584149215
dist3 709.0486584149215
dist1 970.9268767523124
dist2 970.9268767523124
dist3 970.9268767523124
dist1 413.5166260260886
dist2 413.5166260260886
dist3 413.516626